# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298523953237                   -0.85    4.1
  2   -8.300228537156       -2.77       -1.25    1.0
  3   -8.300448768171       -3.66       -1.89    4.0
  4   -8.300462858164       -4.85       -2.78    2.4
  5   -8.300464418980       -5.81       -3.10    3.9
  6   -8.300464520333       -6.99       -3.26    7.9
  7   -8.300464579070       -7.23       -3.40    1.8
  8   -8.300464611433       -7.49       -3.54    1.9
  9   -8.300464634593       -7.64       -3.73    1.0
 10   -8.300464638809       -8.38       -3.84    1.9
 11   -8.300464642820       -8.40       -4.05    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64193437155                   -0.70    5.4
  2   -16.64600480326       -2.39       -1.14    1.0
  3   -16.67922264283       -1.48       -1.86    3.8
  4   -16.67928186111       -4.23       -2.67    3.6
  5   -16.67928614169       -5.37       -3.00    5.6
  6   -16.67928620750       -7.18       -3.31    1.8
  7   -16.67928622088       -7.87       -3.78    1.0
  8   -16.67928622390       -8.52       -4.49    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32599292509                   -0.56    6.4
  2   -33.33284949948       -2.16       -1.00    1.0
  3   -33.33598613329       -2.50       -1.69    4.0
  4   -33.33616666170       -3.74       -2.58    3.4
  5   -33.33693053881       -3.12       -2.73    9.1
  6   -33.33694107284       -4.98       -3.05   10.0
  7   -33.33694390714       -5.55       -4.08    2.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298517157506                   -0.85    4.6
  2   -8.300278533798       -2.75       -1.59    1.0
  3   -8.300445909180       -3.78       -2.51    6.5
  4   -8.300360548703   +   -4.07       -2.25   13.0
  5   -8.300464483835       -3.98       -3.66    4.2
  6   -8.300464609018       -6.90       -4.03    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32555107891                   -0.56    6.1
  2   -33.30623137154   +   -1.71       -1.26    1.0
  3   -6.690320402421   +    1.43       -0.35    8.4
  4   -33.32714473776        1.43       -1.89    7.2
  5   -33.20665441561   +   -0.92       -1.34    4.5
  6   -32.79600410071   +   -0.39       -1.20    6.2
  7   -33.27902717765       -0.32       -1.65    5.1
  8   -33.33600395242       -1.24       -2.31    3.6
  9   -33.33541215844   +   -3.23       -2.23    2.6
 10   -33.33675860837       -2.87       -2.76    2.1
 11   -33.33690193392       -3.84       -3.08    2.0
 12   -33.33693645409       -4.46       -3.51    3.0
 13   -33.33694284101       -5.19       -3.68    3.4
 14   -33.33694337544       -6.27       -3.96    3.1
 15   -33.33694351702       -6.85       -4.24    2.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.